In [773]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html, Dash, State, ctx
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

from dash.exceptions import PreventUpdate


mapbox_token = 'pk.eyJ1IjoiYW5kcmVhc29zdGVkIiwiYSI6ImNsbmJxMjFndDA4dm8ybXJrMzhia2NqdnoifQ.fXDNIJ1LelhA1ypNiaJE9w'

Data wrangling

In [774]:
df = pd.read_csv('Data/homicide_data.csv')
citydata = pd.read_csv('Data/us-cities-top-1k.csv')
statedata = pd.read_csv('Data/states.csv')

In [775]:
citydata=pd.merge(citydata, statedata, on = "State", how = "left")
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('city')

df['reported_date'] = pd.to_datetime(df['reported_date'], format='%Y%m%d')

# Extract month from date and convert to string
df['month'] = df['reported_date'].dt.strftime('%Y-%m')
df['quarter'] = df['reported_date'].dt.to_period('Q').dt.strftime('%Y-Q%q')

df['year'] = df['reported_date'].dt.strftime('%y')
df['month2'] = df['reported_date'].dt.strftime('%m')

#join on data about cities
df = pd.merge(df, citydata.rename(columns={"lat": "avg_lat", "lon": "avg_lon","Abbreviation":"state","State":"Long_state"}), on = ['city',"state"], how = 'left')


df['years_range'] = df.groupby('city')['reported_date'].transform(lambda x: (x.max() - x.min()).days / 365)
df['No_homicides_norm_pr1000'] = (df.groupby('city')['city'].transform('count'))/(df["Population"]*df['years_range'])*1000




avg_city_coords = df[['city','avg_lat','avg_lon','No_homicides_norm_pr1000','Population','state']].drop_duplicates().reset_index()
avg_city_coords = avg_city_coords.drop(49)

In [776]:
obs = df.groupby(['disposition', 'city']).size().reset_index(name='n_obs')

# Pivot the DataFrame to have 'disposition' as columns and 'city' as index
obs_pivot = obs.pivot(index='city', columns='disposition', values='n_obs')

# Calculate the proportion of 'Open/No arrest' over the sum of all dispositions
obs_pivot['open_prop'] = obs_pivot['Open/No arrest'] / obs_pivot.sum(axis=1)

# Reset index if you want to have 'city' as a regular column
obs_pivot = obs_pivot.reset_index()


city_df =pd.merge(avg_city_coords,obs_pivot[["city","open_prop"]],on="city",how="left")

Dash app

In [777]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

cityoptions = [{'label': 'Select all', 'value': 'all cities'}]+[{'label': city, 'value': city} for city in sorted(df['city'].unique())]
raceoptions = [{'label': 'Select all', 'value': 'all'}]+[{'label': r, 'value': r} for r in sorted(df['victim_race'].unique())]
sexoptions = [{'label': 'Select all', 'value': 'all'}]+[{'label': s, 'value': s} for s in sorted(df['victim_sex'].unique())]

# Define layout
# Define layout
app.layout = html.Div([
    html.H4(id='title-output',style={"margin-bottom":"5px","margin-top":"1px","font-size":"24px","text-align":"center"}),
    html.Hr(style={"margin":"0px","marginBottom":"4px"}),
    html.Div([
    dcc.Dropdown(
        id='city_dropdown',
        options = cityoptions,
        value = 'all cities', #default
        #value = None,
        #placeholder = "Select city",
        clearable=False,
        style={"width": "200px", "display": "inline-block","margin-top":"0"}
    ),
        dcc.Dropdown(
        id='race_dropdown',
        options = raceoptions,
        value = 'all', #default
        #value = None,
        #placeholder = "Select race",
        clearable=False,
        style={"width": "200px", "display": "inline-block","margin-top":"0"}
    ),
        dcc.Dropdown(
        id='sex_dropdown',
        options = sexoptions,
        value = 'all', #default
        #value = None,
        #placeholder = "Select gender",
        clearable=False,
        style={"width": "200px", "display": "inline-block","margin-top":"0"}
    )]),
    dbc.Card([dcc.Graph(id = 'murder-map',style={"height":"47.8vh"})]
             ,style={"width": "60%","height":"48vh","display":"inline-block","marginRight":"20px","marginBottom":"5px"}),
    dbc.Card([dcc.Graph(id = 'time-spiral',style={"height":"47.8vh"})],style={"width": "35%","height":"48vh","display":"inline-block","marginBottom":"5px"}),
    dbc.Card([dcc.Graph(id = 'disp-colchart',style={"height":"37.8vh"})],style={"width": "30%","height":"38vh","marginRight":"20px","marginTop":"5px","display":"inline-block"}),
    dbc.Card([dcc.Graph(id = 'sankey',style={"height":"37.8vh"})],style={"width": "65%","height":"38vh","marginTop":"5px","display":"inline-block"}),
dcc.Store(id='selected-city', data = 'all cities')
],
style={"marginLeft":"50px"})


def make_colchart(df):
       
    # Get value counts and reset index
    value_counts = df["disposition"].value_counts().reset_index().sort_values('disposition', ascending = False)
    
    fig = go.Figure(data=go.Bar(x=value_counts['disposition'], 
                                y=value_counts['count'],
                                marker_color="#bd0026")
                                )
    #fig.update_layout(dragmode='zoom',width=500, height=290)
    fig.update_layout(margin={"r": 5, "t": 5, "l": 5, "b": 5}) 
    fig.update_layout(autosize = True,template="plotly_white")
    return fig



def make_spiral_plot(df):
    df['year'] = df['reported_date'].dt.strftime('%y')
    df['month2'] = df['reported_date'].dt.strftime('%m')
    df_summary = df.groupby(['year', 'month2']).size().reset_index(name = 'num_murders_year_month')#.sort_values(['year', 'month2'])
    
    ## håndterer fejl som ellers opstår ved valg af city med klik på stort kort
    if df_summary.empty:
        # Return a default or empty figure
        fig = px.bar_polar()
        return fig

    #quantiles = df_summary['num_murders_year_month'].agg('quantile', q = [0, 0.25, 0.50, 0.75, 1])
    q0, q0_25, q0_5, q0_75, q1 = np.percentile(df_summary['num_murders_year_month'], [0, 25, 50, 75, 100])
    #q0, q0_33, q0_66,q1 = np.percentile(df_summary['num_murders_year_month'], [0, 33, 66, 100])

    all_years = df['year'].unique()
    all_months = df['month2'].unique()
    all_combinations = pd.DataFrame([(year, month) for year in all_years for month in all_months], columns=['year', 'month2']).sort_values(['year', 'month2'])

    # Merge with df_summary to include zeros for missing combinations
    df_summary = pd.merge(all_combinations, df_summary, on=['year', 'month2'], how='left').fillna(0)#replace(0, np.nan)
    df_summary['constant_value'] = 1


    df_summary['hovertemplate'] = df_summary.apply(lambda row: 'Year: %s<br>Month: %s<br>Murders: %d' % (row['year'], row['month2'], row['num_murders_year_month'])
                                                  if row['num_murders_year_month'] > 0 else '',
                                                  axis=1)


    # Create boolean masks for each quantile
#    mask_q0 = (df_summary['num_murders_year_month'] ==q0)
#    mask_q0_25 = (df_summary['num_murders_year_month'] == q0_25)
#    mask_q0_5 = (df_summary['num_murders_year_month'] == q0_5)
#    mask_q0_75 = (df_summary['num_murders_year_month'] == q0_75)
#    mask_q1 = (df_summary['num_murders_year_month'] == q1)
#
#    qs=[q0,q0_25,q0_5,q0_75,q1]

    # Filter the DataFrame based on quantiles
#    points_in_q0 = df_summary[mask_q0]
#    points_in_q0_25 = df_summary[mask_q0_25]
#    points_in_q0_5 = df_summary[mask_q0_5]
#    points_in_q0_75 = df_summary[mask_q0_75]
#    points_in_q1 = df_summary[mask_q1]
#
#    listq=[points_in_q0,points_in_q0_25,points_in_q0_5,points_in_q0_75,points_in_q1]
#    listnozero=[]
#    for i in range(len(listq)):
#        if len(listq[i])!=0:
#            listnozero.append(qs[i])
#
#   colorscale=[]
#   if len(listnozero)==4:
#       colorscale=[(0, 'rgba(0,0,0,0)'), #hide zero counts
#                                              (0, '#ffffb2'),
#                                              (q0/q1, '#fed976'), 
#                                              (q0_25/q1, '#feb24c'),
#                                              (q0_5/q1, '#fd8d3c'),
#                                              (q0_75/q1, '#f03b20'),
#                                              (1, '#bd0026')]
#   
#       
#   elif len(listnozero)==3:
#       colorscale=[(0, 'rgba(0,0,0,0)'), #hide zero counts
#                                              (0, '#ffffb2'),
#                                              (listnozero[0]/q1, '#fecc5c'), 
#                                              (listnozero[1]/q1, '#fd8d3c'),
#                                              (listnozero[2]/q1, '#f03b20'),
#                                              (1, '#bd0026')]
#       
#   elif len(listnozero)==2:
#       colorscale=[(0, 'rgba(0,0,0,0)'), #hide zero counts
#                                              (0, '#ffffb2'),
#                                              (listnozero[0]/q1, '#fecc5c'), 
#                                              (listnozero[1]/q1, '#fd8d3c'),
#                                              (1, '#e31a1c')]
#
#   elif len(listnozero)==1:
#       colorscale=[(0, 'rgba(0,0,0,0)'), #hide zero counts
#                                              (0, '#ffeda0'),
#                                              (listnozero[0]/q1, '#feb24c'),
#                                              (1, '#f03b20')]
        
  #  else:
  #      colorscale=[(0, 'rgba(0,0,0,0)'), #hide zero counts
  #                                             (0, '#ffeda0'),
  #                                             (q0/q1, '#f03b20')]
        
    fig = px.bar_polar(df_summary, 
                       r = 'constant_value', 
                       custom_data=['year', 'month2', 'num_murders_year_month'], #inkluder evt year og month formateret rigtigt i df_summary 
                       theta='month2', 
                       color = 'num_murders_year_month',  
                       color_continuous_scale=[(0, 'rgba(0,0,0,0)'), #hide zero counts
                                               (0, '#ffffb2'),
                                               (q0/q1, '#fed976'), 
                                               (q0_25/q1, '#feb24c'),
                                               (q0_5/q1, '#fd8d3c'),
                                               (q0_75/q1, '#f03b20'),
                                               (1, '#bd0026')],
                       range_color = [q0, q1],
                       labels={"num_murders_year_month": "# murders per<br> year and month"},
                       direction = 'counterclockwise',
                       category_orders={'month2': np.sort(df_summary['month2'].unique()),
                                        'year': np.sort(df_summary['year'].unique())}  # Specify the order of months
                   )
    fig.update_polars(hole = 0.25, radialaxis=dict(showticklabels=False, ticks='', linewidth=0)) 
    #fig.update_traces(hovertemplate='Year: %{customdata[0]}<br>Month: %{customdata[1]}<br>Murders: %{customdata[2]}') 
    fig.update_traces(hovertemplate=df_summary['hovertemplate'])
    fig.update_traces(hoverinfo = 'none')
    fig.update_layout(autosize = True)

    return fig


def make_sankey(df, cat_cols=['disposition','victim_sex','victim_race','age_binned'], value_cols = 'n_obs'):
    df['victim_age'].mask(df['victim_age'] == 'Unknown', -2, inplace=True)
    mask = df['victim_age'] == 'Unknown'
    df = df[~mask]
    df['victim_sex'].mask(df['victim_sex'] == 'Unknown', 'Unknown sex', inplace=True)
    df['victim_race'].mask(df['victim_race'] == 'Unknown', 'Unknown race', inplace=True)
    df['victim_age'] = pd.to_numeric(df['victim_age'])
    df['age_binned'] = pd.cut(x=df['victim_age'], bins=[-3,-1,9,19, 29, 39, 49, 59, 69, 79, 89, 99, 109], labels=['Unknown age','0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99','100+'])
    df = df.sort_values(by=['victim_age'], ascending=True)
    df = df.groupby(['disposition','victim_race','victim_sex','age_binned']).size().reset_index(name='n_obs')
    zeromask = df['n_obs'] == 0
    df = df[~zeromask]

    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#bd0026']*10
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        if catCol=='age_binned':
            labelListTemp = ['Unknown age','0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99','100+']
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    del labelList[12:23]
    labelList = list(dict.fromkeys(labelList))
    labelList = labelList + ['Unknown age','0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99','100+']
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()

    dispList = list(df['disposition'].unique())
    genderList = list(df['victim_sex'].unique())
    num_gender = len(genderList)
    raceList = list(df['victim_race'].unique())
    num_race = len(raceList)
    ageList = list(df['age_binned'].unique())

    presentList = dispList + genderList + raceList + ageList
    

    x = [0.001, 0.001, 0.001, 0.33, 0.33, 0.33, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999]
    y = [0.2, 0.5, 0.8, 0.8, 0.6, 0.2, 0.4, 0.6, 0.7, 0.3, 0.8, 0.9, 0.999, 0.001, 1.5/11, 2.5/11, 4/11, 5/11, 6/11, 7/11, 0.7, 0.75, 0.8, 0.9]

    deleteList = []

    allList = ['Closed by arrest','Open/No arrest', 'Closed without arrest', 'Male', 'Female', 'Unknown sex', 'Black', 'Hispanic', 'White', 'Other', 'Asian', 'Unknown race','100+','Unknown age','0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']
    
    for i in range(len(allList)):
        if allList[i] not in presentList:
            deleteList.append(i)
    

    for j in range(len(deleteList)):
        del allList[deleteList[j]-j]
        del y[deleteList[j]-j]
        del x[deleteList[j]-j]

    if num_gender == 1:
        idx = allList.index(genderList[0])
        y[idx] = 0.5
    
    if num_race == 1:
        idx = allList.index(raceList[0])
        y[idx] = 0.5

    #new stuff
    sourceTargetDf['source'] = pd.Categorical(sourceTargetDf['source'], allList)
    sourceTargetDf['target'] = pd.Categorical(sourceTargetDf['target'], allList)
    sourceTargetDf.sort_values(['source', 'target'], inplace = True)
    sourceTargetDf.reset_index(drop=True)
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))

    

    fig = go.Figure(data=[go.Sankey(
        arrangement = "snap",
    node = dict(
        pad = 15,
        thickness = 20,
        x = x,
        y = y,
        line = dict(
        color = "black",
        width = 0.5
        ),
        color = colorList,
        label = labelList
    ),
    link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        ))],
        layout=dict(
    margin=dict(l=48, r=48, b=45, t=10),
    height=378)
        )
       
    #fig = go.sankey(dict(data=[data], layout=layout))

    return fig

def make_map(df, city):
    jitter_factor = 0.0005  # Adjust the jitter factor as needed
    df['lat'] = df['lat'] + np.random.uniform(-jitter_factor, jitter_factor, len(df))
    df['lon'] = df['lon'] + np.random.uniform(-jitter_factor, jitter_factor, len(df))

    fig = px.scatter_mapbox(
        df if city != 'all cities' else city_df,
        lat="lat" if city != 'all cities' else 'avg_lat',
        lon="lon" if city != 'all cities' else 'avg_lon',
        hover_name="victim_full_name" if city != 'all cities' else 'city',
        hover_data=["victim_race", "victim_age", "victim_sex", "reported_date"] if city != 'all cities' else None,
        custom_data = ['disposition', 'lat', 'lon', 'victim_race', 'victim_age', 'victim_sex', 'reported_date'] if city != 'all cities'\
            else ['No_homicides_norm_pr1000', 'open_prop'],

        color = "disposition"  if city != 'all cities' else "open_prop",
        opacity=0.65 if city != 'all cities' else None,
        color_continuous_scale="YlOrRd" if city=='all cities' else None, #"Rdbu_r" 
        labels={"open_prop": "% Open cases"},
        #labels = hover_labels,
        size='No_homicides_norm_pr1000' if city=='all cities' else None,
        center={"lat": 39.8283-2, "lon": -98.5795} if city=='all cities' else None,
        range_color=[min(city_df["open_prop"])-0.15,max(city_df["open_prop"])] if city=='all cities' else None,
        color_continuous_midpoint=0.5 if city=='all cities' else None,
        category_orders={"disposition": ['Open/No arrest', 'Closed without arrest', 'Closed by arrest']} if city != 'all cities' else None,
        color_discrete_sequence=['#EF553B', '#00CC96','#636EFA']
    )

    
    fig.update_layout(mapbox_style="light", 
                      mapbox_accesstoken=mapbox_token, 
                      #title = title,
                      #mapbox_center={"lat": center_lat, "lon": center_lon},  
                      #mapbox_zoom = 10,
                      autosize = True
                      )
    
    
    
    fig.update_layout(mapbox_zoom = 9.5 if city != 'all cities' else 3.2)

    fig.update_traces(uirevision='persist')


    #['disposition', 'lat', 'lon', 'victim_race', 'victim_age', 'victim_sex', 'reported_date']
    if city == 'all cities':
        fig.update_traces(hovertemplate = "<b>%{hovertext}</b> <br> # Murders per 1000 capita per year: %{customdata[0]:.3f} <br>Open cases: %{customdata[1]:.3f}")
    else:
        fig.update_traces(hovertemplate = "<b>%{hovertext}</b> <br> Lat: %{customdata[1]:.3f} <br> Lon: %{customdata[2]:.3f} <br> Race: %{customdata[3]} <br> Age: %{customdata[4]} <br> Gender: %{customdata[5]} <br> Reported date: %{customdata[6]}")


    
    #fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.update_layout(margin={"r": 0, "l": 15, "b": 5,"t":5})
    fig.update_layout(clickmode='event+select',
                      hovermode='closest')

    return fig



#helper function to filter data based on city, race, sex
def filter_df(selected_city, selected_race = 'all', selected_sex = 'all', df = df):
    all_cities_filter = selected_city == 'all cities'
    all_race_filter = selected_race == 'all'
    all_sex_filter = selected_sex == 'all'

    if all_cities_filter and all_race_filter and all_sex_filter:
        # Handle the case when all filters are 'all'
        filtered_df = df  # Include all data
    else:
        city_filter = df['city'] == selected_city if not all_cities_filter else True
        race_filter = df['victim_race'] == selected_race if not all_race_filter else True
        sex_filter = df['victim_sex'] == selected_sex if not all_sex_filter else True

        filtered_df = df[city_filter & race_filter & sex_filter]
    return filtered_df






@app.callback(
    [Output('murder-map', 'figure'),
    Output('disp-colchart', 'figure'),
    Output('time-spiral', 'figure'),
    Output('sankey', 'figure')
    ],
    
    [Input('city_dropdown', 'value'),
     Input('race_dropdown', 'value'),
     Input('sex_dropdown', 'value')
    ]
)
def update_figs_on_dropdowns(selected_city, selected_race, selected_sex):
    filtered_df = filter_df(selected_city, selected_race, selected_sex)

    fig_map = make_map(filtered_df, selected_city)
    fig_col = make_colchart(filtered_df)
    fig_spiral = make_spiral_plot(filtered_df)
    #fig_map.update_layout(transition={'duration': 500, 'easing': 'cubic-in-out'})
    fig_sankey = make_sankey(filtered_df)
    return fig_map, fig_col, fig_spiral, fig_sankey


@app.callback(
    [Output('disp-colchart', 'figure', allow_duplicate = True),
     Output('time-spiral', 'figure', allow_duplicate = True),
     Output('sankey', 'figure', allow_duplicate = True)
     ], 
    [Input('murder-map', 'selectedData')],
    [State('city_dropdown', 'value'), State('race_dropdown', 'value'), State('sex_dropdown', 'value')],
    prevent_initial_call = True
)
def update_figures_on_map_selection(selected_data, selected_city, selected_race, selected_sex):
    if selected_data:
        # Extract the selected points from the map
        selected_points = selected_data['points']

        # Get the victim_full_name(s) from the selected points
        selected_names = [point['hovertext'] for point in selected_points]

        # Filter the DataFrame based on selected names, city, race, and sex
        filtered_df = df[df['victim_full_name'].isin(selected_names)]
        filtered_df = filter_df(selected_city, selected_race, selected_sex, filtered_df)

        # Generate new figs based on filtered data
        fig_col = make_colchart(filtered_df)
        fig_spiral = make_spiral_plot(filtered_df)
        fig_sankey = make_sankey(filtered_df)
        return fig_col, fig_spiral, fig_sankey
    else:
        fig_col = make_colchart(filter_df(selected_city, selected_race, selected_sex))
        fig_spiral = make_spiral_plot(filter_df(selected_city, selected_race, selected_sex))
        fig_sankey = make_sankey(filter_df(selected_city, selected_race, selected_sex))
        return fig_col, fig_spiral, fig_sankey
    



@app.callback(
    Output('city_dropdown', 'value'),
    Input('selected-city', 'data'),
    prevent_initial_call = True
)
def update_city_dropdown(selected_city):
    return selected_city

@app.callback(
    Output('selected-city', 'data'),
    [Input('murder-map', 'clickData')],
    [State('city_dropdown', 'value')]
)
def update_on_click(click_data, selected_city):
    if selected_city == 'all cities':
        if click_data:
            selected_city = click_data['points'][0]['hovertext']
            return selected_city
    raise PreventUpdate


##### work in progress
@app.callback(
    [Output('murder-map', 'figure', allow_duplicate=True),
     Output('disp-colchart', 'figure', allow_duplicate = True),
     Output('sankey', 'figure', allow_duplicate = True)], 
    [Input('time-spiral', 'selectedData')],
    [State('city_dropdown', 'value'), State('race_dropdown', 'value'), State('sex_dropdown', 'value')],
    prevent_initial_call = True
)
def update_figs_on_spiral_select(selected_data, selected_city, selected_race, selected_sex):
    if selected_data: 
        d = selected_data['points']

        years = np.unique([y for y,m,n in [c['customdata'] for c in d]]).tolist()
        months = np.unique([m for y,m,n in [c['customdata'] for c in d]]).tolist()
        

        filtered_df = df[df['month2'].isin(months) & df['year'].isin(years)]
        filtered_df = filter_df(selected_city, selected_race, selected_sex, filtered_df)

        fig_map = make_map(filtered_df, selected_city)
        fig_col = make_colchart(filtered_df)
        fig_sankey = make_sankey(filtered_df)
        return fig_map, fig_col, fig_sankey
    else:
        filtered_df = filter_df(selected_city, selected_race, selected_sex)

        fig_map = make_map(filtered_df, selected_city)
        fig_col = make_colchart(filtered_df)
        fig_sankey = make_sankey(filtered_df)
        return fig_map, fig_col, fig_sankey

#####


@app.callback(
    Output('title-output', 'children'),
    [Input('city_dropdown', 'value')]
)
def update_title(city):
    city_list = [city for city in df['city'].unique()]
    if city in city_list:
        return f'Murders in {city}' 
    else:
        return 'Murders in 50 large cities in the US'


if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode = "tab", port = 8050)


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

Testing things

Problem: På stort kort opdaterer mord per 1000 indbygger og prop open ikke ved filtrering på race og køn. 
 - Mord per 1000 indbygger er svært at håndtere, da vi ikke har population per befolkningsgruppe
 - Kan man nøjes med blot at opdatere prop closed baseret på filtrering?

#### Num murders per year barchart

In [778]:
make_sankey(filter_df('Las Vegas'),['disposition','victim_sex','victim_race','age_binned'],'n_obs')

C:\Users\clara\AppData\Local\Temp\ipykernel_18572\3833635549.py:182: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\clara\AppData\Local\Temp\ipykernel_18572\3833635549.py:190: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'arrangement': 'snap',
              'link': {'source': array([ 2,  2,  2,  1,  1,  1,  0,  0,  0,  4,  4,  4,  4,  4,  3,  3,  3,  3,
                                         5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6, 11, 11, 11, 11, 11, 11,
                                        11, 11, 11, 11,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,
                                         8, 10, 10, 10, 10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  9,  9,  9],
                                       dtype=int64),
                       'target': array([ 4,  3,  5,  4,  3,  5,  4,  3,  5,  6, 11,  7,  8, 10,  6, 11,  7, 10,
                                        10,  9, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 13, 14, 15, 16, 17, 18,
                                        19, 20, 21, 23, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 15, 16, 17,
                                        18, 14, 15, 16, 17, 18, 19, 20, 21, 22, 13, 14, 16, 17, 18, 19, 20],
                                       dtype=int64),
                       'value': array([594, 208,   7, 342,  52,   3,  43,  73,  59, 376, 265, 304,   7,  27,
                                        76,  77, 153,  27,   1,  68,  34,  50, 165, 104,  62,  20,  12,   3,
                                         1,   1,   2,  18,  45, 100,  85,  66,  13,  11,   1,   1,   1,  19,
                                        26,  75,  65,  84, 102,  46,  25,  10,   4,   1,   1,   4,   1,   5,
                                         4,  11,   9,   7,  12,   2,   3,   2,   7,   3,  19,  15,  11,  11,
                                         2], dtype=int64)},
              'node': {'color': [#bd0026, #bd0026, #bd0026, #bd0026, #bd0026,
                                 #bd0026, #bd0026, #bd0026, #bd0026, #bd0026,
                                 #bd0026, #bd0026, #bd0026, #bd0026, #bd0026,
                                 #bd0026, #bd0026, #bd0026, #bd0026, #bd0026,
                                 #bd0026, #bd0026, #bd0026, #bd0026],
                       'label': [Closed without arrest, Open/No arrest, Closed by
                                 arrest, Female, Male, Unknown sex, Black, White,
                                 Other, Unknown race, Asian, Hispanic, 100+,
                                 Unknown age, 0-9, 10-19, 20-29, 30-39, 40-49,
                                 50-59, 60-69, 70-79, 80-89, 90-99, 100+],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20,
                       'x': [0.001, 0.001, 0.001, 0.33, 0.33, 0.33, 0.66, 0.66,
                             0.66, 0.66, 0.66, 0.66, 0.999, 0.999, 0.999, 0.999,
                             0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999],
                       'y': [0.2, 0.5, 0.8, 0.8, 0.6, 0.2, 0.4, 0.6, 0.7, 0.3,
                             0.8, 0.9, 0.001, 0.13636363636363635,
                             0.22727272727272727, 0.36363636363636365,
                             0.45454545454545453, 0.5454545454545454,
                             0.6363636363636364, 0.7, 0.75, 0.8, 0.9]},
              'type': 'sankey'}],
    'layout': {'height': 378, 'margin': {'b': 45, 'l': 48, 'r': 48, 't': 10}, 'template': '...'}
})

C:\Users\clara\AppData\Local\Temp\ipykernel_18572\3833635549.py:190: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\clara\AppData\Local\Temp\ipykernel_18572\3833635549.py:303: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\clara\AppData\Local\Temp\ipykernel_18572\3833635549.py:304: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni